# Circuitos de Rádio-Frequência

## Ex 1: Figura de ruído do estágio de RF de um receptor BLE

O cenário consiste de um celular utilizando tecnologia BLE transmitindo um sinal de Marte. O sinal deve ser recebido pela antena do telescópio Lovell. Qual a figura de ruído máxima do estágio de RF do telescópio Lovell para que seja respeitada uma BER de $ 10^{-3} $ no padrão BLE?

### Dados:

Frequência da portadora (BLE):

$
\begin{align}
f = 2.4 GHz
\end{align}
$

Largura de banda (BLE):

$
\begin{align}
BW = 2 MHz
\end{align}
$

Taxa de transmissão de bits:

$
\begin{align}
R = 1 Mbps
\end{align}
$

Taxa de erro de bit:

$
\begin{align}
BER = 10^{-3}
\end{align}
$

Razão da energia por bit pela densidade espectral de potência do ruído (a partir da da curva $BER$ por $\frac{E_b}{N_o}$ do padrão BLE):

$
\begin{align}
\frac{E_b}{N_o} = 7 dB
\end{align}
$

Distância entre transmissor (Marte) e receptor (Terra):

$
\begin{align}
d = 55 Gm
\end{align}
$

Potência do transmissor:

$
\begin{align}
P_{Tx} = 10 mW
\end{align}
$

Ganho da antena do transmissor:

$
\begin{align}
G_{Tx} = 20 dB
\end{align}
$

Ganho da antena do telescópio Lovell pode ser estimado a partir do ganho de uma antena parabólica:

$
\begin{align}
G_{Rx} &= 10 log \bigg(k \bigg(\frac{\pi d}{\lambda}\bigg)^2\bigg),
\end{align}
$

O cálculo da potência no receptor ($P_{Rx}$) pode ser feito a partir da equação de transmissão de Friss (em dB):

$
\begin{align}
P_{Rx} = P_{Tx} + G_{Tx} + G_{Rx} + 20 log \bigg(\frac{\lambda}{4 \pi d}\bigg)
\end{align}
$

A figura de ruído pode ser obtida a partir das seguintes equações:

$
\begin{align}
&N_o = kT \\
&N = N_o BW \\
&SNR_i = \frac{P_{Rx}}{N} \\
&SNR_{o,min} = \frac{R}{BW}\frac{E_b}{N_o} \\
&NF = \frac{SNR_i}{SNR_{o,min}},
\end{align}
$

onde $N_o$ é a densidade espectral de potência do ruído térmico, $N$ é potência do ruído para a banda do BLE, $SNR_i$ é a relação sinal-ruído na entrada do estágio de RF, $SNR_{o,min}$ é a menor relação sinal-ruído permitida pelo padrão BLE na entrada do processo digital e $NF$ é a figura de ruído máxima permitida ao estágio de RF para que ele esteja adequado ao padrão BLE.

In [39]:
import numpy as np
import scipy.constants as sp

# temperatura de 290 Kelvin
T = 290

# distancia entre receptor e transmissor e' 55 Gigametros
d = 55 * 10**9

# frequencia da portadora
f = 2.4 * 10**9

# largura de banda do BLE
BW = 2 * 10**6

# taxa de transmissao de bits
R = 1 * 10**6

# razao da energia do bit por ruido e' 7 dB
EbNo = 10**(7 / 10)

# potencia no transmissor e' 10 mW = 10 dBm
Ptx = 10 * 10**(-3)
Ptxdb = 10 * np.log10(Ptx / (1 * 10**(-3)))

# ganho da antena do transmissor e' 20 dB
Gtxdb = 20

# razão entre area efetiva e fisica da antena 
k = 0.6

# diametro do telescopio Lovell
di = 76.2

# comprimento de onda da portadora em 2.4 GHz
lambdaa = sp.c / f

In [34]:
# estimativa do ganho da antena do telescopio Lovell
Grxdb = 10 * np.log10(k * ((sp.pi * di / lambdaa) ** 2))

print("Ganho da antena do receptor = {} dB".format(Grx))

Ganho da antena do receptor = 63.43 dB


In [35]:
# calculo da potencia no receptor utilizando a formula de Friss em dB
Prxdb = Ptxdb + Gtxdb + Grxdb + 20 * np.log10(lambdaa / (4 * sp.pi * d))
Prx = 10**(Prxdb / 10) * 10**(-3) # em Watts

print("Potência recebida no receptor = {} dBm".format(Prxdb))

Potência recebida no receptor = -161.42784168581565 dBm


In [40]:
# calculo da densidade espectral de ruido
No = sp.k * T

# calculo da potencia do ruido na banda BLE
N = No * BW

# razao sinal-ruido na entrada do estagio RF
SNRi = Prx / N

# razao sinal-ruido minima na saida do estagio RF
SNRo = R * EbNo / BW

# figura de ruido
NF = SNRi / SNRo

print("Potencia do sinal na entrada = {}".format(Prx))
print("Potencia do ruido na entrada = {}".format(N))
print("Razao sinal ruido na entrada = {}".format(SNRi))
print("Razao sinal ruido na saida = {}".format(SNRo))
print("Figura de ruido = {} dB".format(10*np.log10(NF)))

Potencia do sinal na entrada = 7.19806611570255e-20
Potencia do ruido na entrada = 2.76129704e-20
Razao sinal ruido na entrada = 2.6067699387033527
Razao sinal ruido na saida = 2.505936168136361
Figura de ruido = 0.1713269972817134 dB


### Conclusões

Como é possível observar nos resultados obtidos, a potência que chega ao receptor é inferior a potência do sinal de ruído que chega ao receptor, portanto não é possível atingir a SNR demandada pelo padrão BLE. A única forma disto ocorrer é caso o estágio de RF consiga aumentar a SNR do sistema (NF < 0 dB), o que não é fisicamente factível. Porém é visto pelo resultado da figura de ruído ($NF = -54 dB$) que é isto que seria necessário para atender as especificações do padrão BLE.

Observou-se que reduzindo a temperatura para 0.001 K (próximo do zero absoluto), a potência do ruído baixou o suficiente para que a figura de ruído resultante fosse positiva, tornando o projeto do sistema "factível".